# General approach to kNN

1. Collect: any method
2. Prepare: numeric values are need for a distance calculation. A structured data format is best.
3. Anayze: any method
4. Train: does not apply to the kNN algorithm
5. Test: calculate the error rate
6. Use: This application needs to get some input data and output structure numeric values. Next, the application run the kNN algorithm on this input data and determines whitch class the input data should belong to.

## 1. Collect: 
Tengo el fichero train.csv, test.csv y gender_submission.csv (este último para ver el formato del fichero que se envía).

## 2. Prepare
Hay que preparar los datos primero del fichero de training y luego del de test.
* Extraer las labels.
* Transformar los valores no numericos en valores numéricos.
* Tratar los valores nulos.
* Establecer valores entre 0 y 1

Por simplificar obviaré el control de errores

In [27]:
import pickle

def saveFile(filename, data):
    output = open(filename, 'wb')
    pickle.dump(data, output)
    output.close()
    
def loadFile(filename):
    pkl_file = open(filename, 'rb')

    data = pickle.load(pkl_file)
    pkl_file.close()
    
    return data

def loadData(filename):
    data = loadFile(filename + '_data.txt')
    labels = loadFile(filename + '_labels.txt')
    
    return data, labels

In [6]:
import re


MAX_CLASS = 3 # Max value of passae Class
MAX_AGE = 100
MAX_SIBLINGS = 10
MAX_PARENTS = 10
MAX_TICKET = 3101317.0
MAX_FARE = 512.3292
MAX_CABIN = 148.0

def removeName(line):
    return re.sub('"(.*?)",','', line)

def normalizeClass(classNumber):
    return (float(classNumber) - 1.0)/ (MAX_CLASS -1)

def normalizeSex(sex):
    sexNormalizedValues = { "male" : 0.5, "female" : 1}
    
    if sex in sexNormalizedValues:
        return sexNormalizedValues[sex]
    else:
        return 0

def normalizeAge(age):
    if not age:
        return 0
    else:
        return float(age) / MAX_AGE
    
def normalizeNumberOfSiblings(siblings):
    if not siblings:
        return 0
    else:
        return float(siblings) / MAX_SIBLINGS

def normalizeNumberOfParents(parents):
    if not parents:
        return 0
    else:
        return float(parents) / MAX_PARENTS
    
def normalizePrefixTicket(ticket):
    prefixes = { 'A/5' : 0.01, 'A/5.' : 0.01, 'A./5.': 0.01, 'A.5.' : 0.01, 'PC' : 0.02, 'STON/O2.' : 0.03, 'PP' : 0.04, 'C.A.' : 0.05, 'CA' : 0.05 , 'CA.' : 0.05, 'SC/Paris' : 0.06, 'SC/PARIS' : 0.06, 'S.C./PARIS' : 0.06, 'S.C./A.4.' : 0.07, 'A/4.' : 0.08, 'A4.' : 0.08, 'A/4' : 0.08, 'S.P.' : 0.09, 'S.O.C.' : 0.10, 'SO/C' : 0.10, 'W./C.' : 0.11, 'W/C' : 0.11, 'SOTON/OQ' : 0.12, 'SOTON/O.Q.' : 0.12, 'W.E.P.' : 0.13, 'WE/P' : 0.13, 'STON/O' : 0.14, 'C' : 0.15, 'S.O.P.' : 0.16, 'Fa' : 0.17, 'F.C.C.' : 0.18, 'SW/PP' : 0.19, 'S.W./PP' : 0.19, 'SCO/W' : 0.20, 'P/PP' : 0.21, 'SC': 0.22, 'SC/AH' : 0.23, 'A/S': 0.24, 'S.O./P.P.' : 0.25, 'F.C.' : 0.26, 'SOTON/O2' : 0.27, 'C.A./SOTON' : 0.28 }

    ticketSplitedData = ticket.split(' ')
    if len(ticketSplitedData) > 1:
        ticketPrefix = ticketSplitedData[0]
        if ticketPrefix in prefixes:
            return prefixes[ticketPrefix]
        else:
            return 0
    else:
        return 0

def normalizeTicketNumber(ticket):
    return getTicketNumber(ticket) / MAX_TICKET

def getTicketNumber(ticket):
    ticketSplitedData = ticket.split(' ')
    
    if len(ticketSplitedData) == 1:
        if ticket == 'LINE':
            return 0
        else:
            return float(ticket)
    else:
        return float(ticketSplitedData[len(ticketSplitedData)-1]) 
    
def normalizeFare(fare):
    if len(fare) == 0:
        return 0
    else:
        return float(fare) / MAX_FARE
    
def normalizeCabinPrefix(cabin):
    prefixes = { 'A' : 0.1, 'B' : 0.2, 'C': 0.3, 'D' : 0.4, 'E' : 0.5, 'F' : 0.6, 'G' : 0.7, 'T': 0.8 }
    
    if len(cabin) == 0:
        return 0
    else:
        cabinSplitedData = cabin.split(' ')
    
        if len(cabinSplitedData) == 1:
            return prefixes[cabin[0]]
        else:
            return prefixes[cabinSplitedData[0][0]]
        
def normalizeCabinNumber(cabin):
    if len(cabin) == 0:
        return 0
    else:
        cabinNumber = getCabinNumber(cabin)
        if cabinNumber == 0:
            return 0
        else:
            return cabinNumber / MAX_CABIN
        
def getCabinNumber(cabin):
    cabinSplitedData = cabin.split(' ')
    
    if len(cabinSplitedData) == 1:
        if len(cabin) == 1:
            return 0
        else:
            return float(cabin[1:])
    else: 
        if len(cabinSplitedData[0]) == 1:
            return 0
        else:
            return float(cabinSplitedData[0][1:])

def normalizeEmbarked(embarked):
    embarkedOptions = { '' : 0, 'C' : 0.25, 'Q': 0.5, 'S' : 1}
    
    return embarkedOptions[embarked]
        
def processTrainingFile(filename, test):
    file = open(filename)
    
    labels = []
    trainingData = []
    
    # Leer fichero linea a linea
    header = True
    for line in file.readlines():
        if not header:
            line = removeName(line)
 
            if test:
                line = 'fake,' + line
            
            lineValueList = line.strip().split(',')
    
            if not test:
                labels.append(lineValueList[1])
            
            treatedData = []
            treatedData.append(normalizeClass(lineValueList[2]))
            treatedData.append(normalizeSex(lineValueList[3]))
            treatedData.append(normalizeAge(lineValueList[4]))
            treatedData.append(normalizeNumberOfSiblings(lineValueList[5]))
            treatedData.append(normalizeNumberOfParents(lineValueList[6]))
            treatedData.append(normalizePrefixTicket(lineValueList[7]))
            treatedData.append(normalizeTicketNumber(lineValueList[7]))
            treatedData.append(normalizeFare(lineValueList[8]))
            treatedData.append(normalizeCabinPrefix(lineValueList[9]))
            treatedData.append(normalizeCabinNumber(lineValueList[9]))
            treatedData.append(normalizeEmbarked(lineValueList[10]))
            
            trainingData.append(treatedData)
        else:
            header = False
                
    # Guardar el fichero con las transformaciones
    saveFile(filename + '_data.txt', trainingData)
    # Guardar el fichero con las labels
    saveFile(filename + '_labels.txt', labels)

In [49]:
processTrainingFile('titanic/train.csv', False)

In [ ]:
processTrainingFile('titanic/test.csv', True)

In [44]:
def divideTrainingFile(filename, testPercentage):
    # Lee los ficheros, calcula el numero necesario de registros
    data, labels = loadData(filename)
    testItems = int(len(data) * testPercentage)

    # Divide el vector y las matrices
    data_training = data[testItems:]
    label_training = data[testItems:]
    data_test = data[:testItems]
    label_test = labels[:testItems]

    
    # Guarda en ficheros adicionales
    saveFile(filename + '.data_training', data_training)
    saveFile(filename + '.label_training', label_training)
    saveFile(filename + '.data_test', data_test)
    saveFile(filename + '.label_test', label_test)

In [45]:
divideTrainingFile('titanic/train.csv', 0.1)

## 3. Anayze: any method
Necesitaria ver formas de analizar estos datos. Siguiendo con los libros o quizas con cursos de formación

## 4. Train: does not apply to the kNN algorithm
No hay training, sino directamente el algoritmo. En este caso se trata de una modificación del kNN que calcula las distancias para que calcule para cualquier profundidad de los vectores

In [12]:
import numpy
import operator

def classify0(inputVector, trainingDataSet, trainingLabels, k):
    # inX es un vector
    # dataSet es una matriz
    # labels es un vector, el numero de elementos de labels = numero de filas de dataset
    # k es un entero menor que el numero de filas de dataset
    
    # CALCULANDO UNA MATRIZ DE DISTANCIAS ENTRE inX Y LOS PUNTOS EXISTENTES EN dataSet
    # para ello se calcula la distancia Euclidiana con cada uno de los puntos
    # La funcion 'tile' repite el vector inX en una matriz de dataSetSize filas y 1 columna
    # A esa matriz se le resta el data set
    trainingDataSetSize = trainingDataSet.shape[0]
    
    differentialMatrix = np.tile(inputVector, (trainingDataSetSize,1)) - trainingDataSet
    squaredDifferentialMatrix = differentialMatrix**2

    # Results in vectors
    squaredDistances = squaredDifferentialMatrix.sum(axis=1)
    distances = squaredDistances**0.5
    
    #TODO: NECESITO ENTENDER COMO FUNCIONA ESTE argsort
    sortedDistancesIndexes = distances.argsort()
    
    classCount = {}
    
    for i in range(k):
        label = trainingLabels[sortedDistancesIndexes[i]]
        classCount[label] = classCount.get(label,0) + 1
        
    sortedClassCount = sorted(classCount.items(), key=operator.itemgetter(1), reverse=True)
    
    return sortedClassCount[0][0]

## 5. Test: calculate the error rate

In [21]:
import numpy as np



def calculate():
    training, training_labels = loadData('titanic/train.csv')
    test, test_labels = loadData('titanic/test.csv')
    
    np_training = np.array(training)
    
    print(test_labels)
    
    # Recorrer todos los test
    numberOfTests = len(test)
    success = 0
    for i in range(numberOfTests):
        
        ## Ejecutar classify
        classifyResult = classify0(test[i], np_training, training_labels, 3)
        ## Obtener resultado e incrementar el contador de exitosos
        if classifyResult == test_labels[i]:
            success += 1

    # Calcular el error rate = total / exitosos
    return success, success / numberOfTests

## 6. Use

In [46]:
calculate()

#REVIEW los ficheros que se cargan en el calculate

[]


IndexError: list index out of range